## I referred to the following webpages for the implementation.
- Implementation of Transformer<br>
https://qiita.com/halhorn/items/c91497522be27bde17ce<br>
https://github.com/kpot/keras-transformer/tree/master/keras_transformer<br>
https://github.com/Lsdefine/attention-is-all-you-need-keras<br>
- Usage of "\_\_call\_\_" method<br>
https://qiita.com/kyo-bad/items/439d8cc3a0424c45214a

In [1]:
import numpy as np
import math

import tensorflow as tf

from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Layer, Embedding, Input, Reshape, Lambda, Add
from keras import backend as K
from keras.initializers import RandomNormal
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.callbacks import Callback

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or 

In [32]:
vocab_size = 8000
d_model = 512
MAX_LEN = 716
class_num = 9
PAD_ID = 0
warmup_steps = 4000
NUM_TRAIN = 5893
NUM_TEST = 1474
batch_size = 16
epochs = 1500

In [4]:
def parse(example):
    features = tf.parse_single_example(
        example,
        features={
            "X": tf.FixedLenFeature([MAX_LEN], dtype=tf.float32),
            "Y": tf.FixedLenFeature((class_num,), dtype=tf.float32)
        }
    )
    
    X = features["X"]
    Y = features["Y"]
    
    return X, Y

In [5]:
def iterator(tfrecord_path, data_len):
    dataset = tf.data.TFRecordDataset([tfrecord_path]).map(parse)
    dataset = dataset.repeat(-1).batch(data_len)
    iterator = dataset.make_one_shot_iterator()
    
    X, Y = iterator.get_next()
    X = tf.reshape(X, [-1, MAX_LEN])
    
    return X, Y

In [6]:
%%time

x_train, y_train = iterator("train_transformer_with_sp.tfrecord", NUM_TRAIN)
x_test, y_test = iterator("test_transformer_with_sp.tfrecord", NUM_TEST)

x_train = tf.Session().run(x_train)
y_train = tf.Session().run(y_train)
x_test = tf.Session().run(x_test)
y_test = tf.Session().run(y_test)

CPU times: user 2.11 s, sys: 1.38 s, total: 3.49 s
Wall time: 2.49 s


In [7]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((5893, 716), (5893, 9), (1474, 716), (1474, 9))

## Transformer Architecture

In [8]:
class MultiheadAttention():
    ## hidden_dim has to be multiples of head_num
    def __init__(self, max_len, hidden_dim=512, head_num=8, dropout_rate=0.1, *args, **kwargs):
        self.max_len = max_len
        self.hidden_dim = hidden_dim
        self.head_num = head_num
        self.dropout_rate = dropout_rate
        
        self.q_dense_layer = Dense(hidden_dim, use_bias=False)
        self.k_dense_layer = Dense(hidden_dim, use_bias=False)
        self.v_dense_layer = Dense(hidden_dim, use_bias=False)
        self.output_dense_layer = Dense(hidden_dim, use_bias=False)
        self.attention_dropout_layer = Dropout(dropout_rate)
        
    def split_heads(self, x):
        def reshape(x):
            x = tf.reshape(x, [-1, self.max_len, self.head_num, self.hidden_dim // self.head_num])
            return tf.transpose(x, [0, 2, 1, 3])
        
        out = Lambda(reshape)(x)
        return out
    
    def combine_heads(self, heads):
        def reshape(x):
            heads = tf.transpose(x, [0, 2, 1, 3])
            return tf.reshape(x, [-1, self.max_len, self.hidden_dim])
        
        out = Lambda(reshape)(heads)
        return out
        
    def __call__(self, query, memory):
        #two arguments of query and memory are already encoded as embedded vectors for all words
        q = self.q_dense_layer(query)
        k = self.k_dense_layer(memory)
        v = self.v_dense_layer(memory)
        
        q = self.split_heads(q)
        k = self.split_heads(k)
        v = self.split_heads(v)
        
        #for scaled dot-product
        depth_inside_each_head = self.hidden_dim // self.head_num
        q = Lambda(lambda x: x * (depth_inside_each_head ** -0.5))(q)
        
        #q.shape = (batch_size, head_num, query_len, emb_dim)
        #k.shape = (batch_size, head_num, memory_len, emb_dim)
        #batch_dot(q, k).shape = (batch_size, head_num, query_len, memory_len)
        score = Lambda(lambda x: K.batch_dot(x[0], x[1], axes=[3, 3]))([q, k])
        normalized_score = Activation("softmax")(score)
        normalized_score = self.attention_dropout_layer(normalized_score)
        
        #normalized_score.shape = (batch_size, head_num, query_length, memory_length)
        #v.shape = (batch_size, head_num, memory_length, depth)
        #attention_weighted_output.shape = (batch_size, head_num, query_length, depth)
        attention_weighted_output = Lambda(lambda x: K.batch_dot(x[0], x[1], axes=[3, 2]))([normalized_score, v])
        attention_weighted_output = self.combine_heads(attention_weighted_output)
        return self.output_dense_layer(attention_weighted_output)

In [9]:
# SlefAttention class inherits MultiheadAttention class so that it can make query and memory come from the same source.
class SelfAttention(MultiheadAttention):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def __call__(self, query):
        return super().__call__(query, query)

In [10]:
class PositionwiseFeedForwardNetwork():
    
    def __init__(self, hidden_dim, dropout_rate, *args, **kwargs):
        self.hidden_dim = hidden_dim
        self.dropout_rate = dropout_rate
        
        self.first_dense_layer = Dense(hidden_dim*4, use_bias=True, activation="relu")
        self.second_dense_layer = Dense(hidden_dim, use_bias=True, activation="linear")
        self.dropout_layer = Dropout(dropout_rate)
        
    def __call__(self, inputs):
        # make the network more flexible to learn for the first dense layer(non-linear transformation is used),
        # and put the network back into the same hidden dim as original(linear transformation is used)
        x = self.first_dense_layer(inputs)
        x = self.dropout_layer(x)
        return self.second_dense_layer(x)

In [11]:
class LayerNormalization(Layer):
    def __init__(self, axis=-1, **kwargs):
        self.axis = axis
        super(LayerNormalization, self).__init__(**kwargs)
        
    def get_config(self):
        config = super().get_config()
        config["axis"] = self.axis
        return config
    
    def build(self, input_shape):
        hidden_dim = input_shape[-1]
        self.scale = self.add_weight("layer_norm_scale", shape=[hidden_dim],
                                    initializer="ones")
        self.shift = self.add_weight("layer_norm_shift", shape=[hidden_dim],
                                    initializer="zeros")
        super(LayerNormalization, self).build(input_shape)
        
    def call(self, inputs, epsilon=1e-6):
        mean = K.mean(inputs, axis=[-1], keepdims=True)
        variance = K.var(inputs, axis=[-1], keepdims=True)
        normalized_inputs = (inputs - mean) / (K.sqrt(variance) + epsilon)
        return normalized_inputs * self.scale + self.shift
    
    def compute_output_shape(self, input_shape):
        return input_shape

In [12]:
class PreLayerNormPostResidualConnectionWrapper():
    def __init__(self, layer, dropout_rate, *args, **kwargs):
        self.layer = layer
        self.layer_norm = LayerNormalization()
        self.dropout_layer = Dropout(dropout_rate)
        
    def __call__(self, inputs, *args, **kwargs):
        x = self.layer_norm(inputs)
        x = self.layer(x)
        outputs = self.dropout_layer(x)
        results = Add()([inputs, outputs])
        return results

In [13]:
class AddPositionalEncoding(Layer): 
    def call(self, inputs):
        data_type = inputs.dtype
        batch_size, max_len, emb_dim = tf.unstack(tf.shape(inputs))
        # i is from 0 to 255 when emb_dim is 512
        #so the doubled_i is from 0 to 510
        doubled_i = K.arange(emb_dim) // 2 * 2
        exponent = K.tile(K.expand_dims(doubled_i, 0), [max_len, 1])
        denominator_matrix = K.pow(10000.0, K.cast(exponent / emb_dim, data_type))
        
        # since cos(x) = sin(x + π/2), we convert the series of [sin, cos, sin, cos, ...]
        # into [sin, sin, sin, sin, ...]
        to_convert = K.cast(K.arange(emb_dim) % 2, data_type) * math.pi / 2
        convert_matrix = K.tile(tf.expand_dims(to_convert, 0), [max_len, 1])
        
        seq_pos = K.arange(max_len)
        numerator_matrix = K.cast(K.tile(K.expand_dims(seq_pos, 1), [1, emb_dim]), data_type)
        
        positinal_encoding = K.sin(numerator_matrix / denominator_matrix + convert_matrix)
        batched_positional_encoding = K.tile(K.expand_dims(positinal_encoding, 0), [batch_size, 1, 1])
        return inputs + batched_positional_encoding
    
    def compute_output_shape(self, input_shape):
        return input_shape

In [14]:
class MakeZeroPads(Layer):
    def __init__(self, seq_len, vocab_size, emb_dim, data_type="float32", *args, **kwargs):
        self.emb_dim = emb_dim
        super(MakeZeroPads, self).__init__(*args, **kwargs)
        
    def call(self, inputs):
        mask_for_pads = tf.to_float(tf.not_equal(inputs, PAD_ID))
        pads_masked_embedding = inputs * mask_for_pads
        return pads_masked_embedding * (self.emb_dim ** 0.5)
    
    def compute_output_shape(self, input_shape):
        return input_shape

In [15]:
class Encoder():
    def __init__(self, vocab_size, max_len, stack_num, head_num, emb_dim, dropout_rate, *args, **kwargs):
        self.vocab_size = vocab_size
        self.max_len = max_len
        self.stack_num = stack_num
        self.head_num = head_num
        self.emb_dim = emb_dim
        self.dropout_rate = dropout_rate
        
        self.embedding_layer = Embedding(self.vocab_size,
                           self.emb_dim,
                           embeddings_initializer=RandomNormal(mean=0.0, stddev=self.emb_dim**-0.5)
                          )
        self.make_zero_pads_layer = MakeZeroPads(self.max_len, vocab_size, emb_dim)
        self.add_pos_enc_layer = AddPositionalEncoding()
        self.input_dropout_layer = Dropout(dropout_rate)
        
        self.attention_block_list = []
        for _ in range(stack_num):
            self_attention_layer = SelfAttention(self.max_len, self.emb_dim, self.head_num, self.dropout_rate)
            pffn_layer = PositionwiseFeedForwardNetwork(self.emb_dim, self.dropout_rate)
            self.attention_block_list.append([
                PreLayerNormPostResidualConnectionWrapper(self_attention_layer, dropout_rate),
                PreLayerNormPostResidualConnectionWrapper(pffn_layer, dropout_rate)
            ])
        self.output_layer_norm = LayerNormalization()
        
    def __call__(self, inputs):
        x = self.embedding_layer(inputs)
        x = self.make_zero_pads_layer(x)
        x = self.add_pos_enc_layer(x)
        x = self.input_dropout_layer(x)
        
        for i, set_of_layers_list in enumerate(self.attention_block_list):
            self_attention_layer, pffn_layer = tuple(set_of_layers_list)
            x = self_attention_layer(x)
            x = pffn_layer(x)
            
        return self.output_layer_norm(x)

In [16]:
# Transformer classification model
inputs = Input((MAX_LEN,))
transformer_encoder = Encoder(vocab_size=vocab_size, stack_num=6, head_num=8, emb_dim=512, dropout_rate=0.1, max_len=MAX_LEN)
encoder_output = transformer_encoder(inputs)
#Since the task is text classification, we just need the first token for each sample
summarized_vecs = Lambda(lambda x: x[:, 0, :])(encoder_output)
outputs = Dense(class_num)(summarized_vecs)
model = Model(inputs, outputs)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 716)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 716, 512)     4096000     input_1[0][0]                    
__________________________________________________________________________________________________
make_zero_pads_1 (MakeZeroPads) (None, 716, 512)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
add_positional_encoding_1 (AddP (None, 716, 512)     0           make_zero_pads_1[0][0]           
__________________________________________________________________________________________________
dropout_1 

In [16]:
plot_model(model, to_file="transformer_encoder.png", show_shapes=True, show_layer_names=True)

In [17]:
#learning rate for Adam
class LRSchedulerPerStep(Callback):
    def __init__(self, d_model, warmup_steps=4000):
        self.d_model = d_model
        self.warmup_step = warmup_steps
        self.step_num = 0
        
    def on_batch_begin(self, batch, logs=None):
        self.step_num += 1
        updated_lr = self.d_model * min(self.step_num ** (-0.5), self.step_num * (self.warmup_step ** (-1.5)))
        K.set_value(self.model.optimizer.lr, updated_lr)

In [18]:
lr_scheduler = LRSchedulerPerStep(d_model, warmup_steps)

In [19]:
opt = Adam(beta_1=0.9, beta_2=0.98, epsilon=1e-9)

## About customized loss function
https://stackoverflow.com/questions/50063613/add-loss-function-in-keras<br>
https://github.com/kpot/keras-transformer/blob/b9d4e76c535c0c62cadc73e37416e4dc18b635ca/keras_transformer/bert.py#L212<br>
https://github.com/tensorflow/models/blob/b9ef963d1e84da0bb9c0a6039457c39a699ea149/official/transformer/v2/metrics.py#L47<br>

In [20]:
# smoothed loss function
class SmoothedLossSparseCategoricalXEntropy:
    def __init__(self, smoothing, class_num):
        self.smoothing = smoothing
        self.class_num = class_num
        
    def __call__(self, y_true, y_pred):
        confidence = 1.0 - self.smoothing
        low_confidence = (1.0 - confidence) / tf.cast(self.class_num - 1, tf.float32)
        smoothed_labels = tf.one_hot(
            tf.cast(y_true, tf.int32),
            depth=self.class_num,
            on_value=confidence,
            off_value=low_confidence
        )
        xentropy = tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=y_pred,
            labels=smoothed_labels
        )
        
        lowest_loss = -(confidence * tf.log(confidence) + 
                       tf.cast(self.class_num -1, tf.float32) * low_confidence * tf.log(low_confidence + 1e-20))
        final_loss = xentropy - lowest_loss
        return final_loss

In [21]:
y_train = np.argmax(y_train, axis=-1)
y_test = np.argmax(y_test, axis=-1)
y_train.shape, y_test.shape

((5893,), (1474,))

In [24]:
model.compile(optimizer=opt, loss=SmoothedLossSparseCategoricalXEntropy(smoothing=0.1, class_num=class_num))

In [ ]:
model.compile(optimizer=opt, loss=categorical_crossentropy)

In [33]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), callbacks=[lr_scheduler])

Train on 5893 samples, validate on 1474 samples
Epoch 1/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 22.9624 - val_loss: 32.3749
Epoch 2/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 23.7330 - val_loss: 13.7559
Epoch 3/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 23.5895 - val_loss: 16.3020
Epoch 4/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 25.0843 - val_loss: 57.8977
Epoch 5/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 26.2714 - val_loss: 44.8746
Epoch 6/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 23.4282 - val_loss: 15.9615
Epoch 7/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 25.4451 - val_loss: 19.3501
Epoch 8/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 21.4507 - val_loss: 17.5041
Epoch 9/1500
5893/5893 [==============================] - 133s 23ms/step - loss:

5893/5893 [==============================] - 133s 23ms/step - loss: 20.8964 - val_loss: 25.4310
Epoch 76/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 24.9266 - val_loss: 39.4387
Epoch 77/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 22.8916 - val_loss: 30.0352
Epoch 78/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 18.8790 - val_loss: 25.6923
Epoch 79/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 23.0707 - val_loss: 22.3227
Epoch 80/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 21.2750 - val_loss: 22.3666
Epoch 81/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 21.2315 - val_loss: 18.0864
Epoch 82/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 22.2467 - val_loss: 35.6662
Epoch 83/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 22.2531 - val_loss: 32.9786
Epoch 84/1500
5893/5893 

5893/5893 [==============================] - 133s 23ms/step - loss: 20.2066 - val_loss: 14.3196
Epoch 150/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 25.3517 - val_loss: 17.1741
Epoch 151/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 20.2528 - val_loss: 15.3556
Epoch 152/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 28.5929 - val_loss: 25.5474
Epoch 153/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 19.7322 - val_loss: 10.4462
Epoch 154/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 22.2833 - val_loss: 15.7129
Epoch 155/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 19.6267 - val_loss: 21.5379
Epoch 156/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 26.2649 - val_loss: 51.8340
Epoch 157/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 28.3988 - val_loss: 14.4192
Epoch 158/1500
5

Epoch 223/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 22.2639 - val_loss: 28.3779
Epoch 224/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 20.7945 - val_loss: 25.9436
Epoch 225/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 18.8163 - val_loss: 23.9911
Epoch 226/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 22.4190 - val_loss: 18.8584
Epoch 227/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 18.7762 - val_loss: 26.3570
Epoch 228/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 20.7953 - val_loss: 21.7330
Epoch 229/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 24.1314 - val_loss: 43.0261
Epoch 230/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 21.8534 - val_loss: 20.8807
Epoch 231/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 20.6378 - val_loss: 21.7428
E

Epoch 297/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 17.3048 - val_loss: 41.1021
Epoch 298/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 21.2418 - val_loss: 27.4605
Epoch 299/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 19.3997 - val_loss: 31.3019
Epoch 300/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 20.5544 - val_loss: 25.5013
Epoch 301/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 28.7757 - val_loss: 22.8332
Epoch 302/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 22.9393 - val_loss: 28.9404
Epoch 303/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 19.7574 - val_loss: 35.4959
Epoch 304/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 22.1593 - val_loss: 22.4182
Epoch 305/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 21.3106 - val_loss: 14.3136
E

...

Epoch 593/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 15.6430 - val_loss: 43.8835
Epoch 594/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 18.7378 - val_loss: 19.8704
Epoch 595/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 16.8682 - val_loss: 12.2706
Epoch 596/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 19.9721 - val_loss: 15.6397
Epoch 597/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 19.6423 - val_loss: 16.6528
Epoch 598/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 17.0407 - val_loss: 14.2894
Epoch 599/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 18.5584 - val_loss: 26.5086
Epoch 600/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 16.2116 - val_loss: 13.1457
Epoch 601/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 15.7128 - val_loss: 18.2923
E

Epoch 667/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 20.0326 - val_loss: 18.7542
Epoch 668/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 19.7447 - val_loss: 23.2304
Epoch 669/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 16.5712 - val_loss: 40.6428
Epoch 670/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 18.3142 - val_loss: 29.2735
Epoch 671/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 16.8980 - val_loss: 38.3935
Epoch 672/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 18.3204 - val_loss: 15.5841
Epoch 673/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 17.9696 - val_loss: 12.7203
Epoch 674/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 18.0325 - val_loss: 8.3871
Epoch 675/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 16.6565 - val_loss: 20.1963
Ep

Epoch 741/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 16.3864 - val_loss: 23.2348
Epoch 742/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 15.0170 - val_loss: 18.0799
Epoch 743/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 19.9749 - val_loss: 24.5515
Epoch 744/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 18.9259 - val_loss: 21.5562
Epoch 745/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 18.4928 - val_loss: 20.9698
Epoch 746/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 18.1134 - val_loss: 28.0166
Epoch 747/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 17.8244 - val_loss: 19.1329
Epoch 748/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 17.4559 - val_loss: 23.7582
Epoch 749/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 16.9105 - val_loss: 16.3236
E

Epoch 815/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 18.2793 - val_loss: 14.8239
Epoch 816/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 18.3668 - val_loss: 26.8804
Epoch 817/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 17.1494 - val_loss: 30.3411
Epoch 818/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 17.4518 - val_loss: 16.0064
Epoch 819/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 15.4407 - val_loss: 11.9828
Epoch 820/1500
5893/5893 [==============================] - 133s 23ms/step - loss: 17.4047 - val_loss: 23.0878
Epoch 821/1500
2192/5893 [==========>...................] - ETA: 1:19 - loss: 15.4437

KeyboardInterrupt: 